# Action2	"多辆车的路径规划 VRP：

条件：经过中国33个城市，一共4辆车，每辆车最大行驶10000公里

目标：使得每辆车的行驶里程数更接近

需要注意：

1）在VRP问题中，路径上给点赋的index和点实际的index不一样，需要使用IndexToNode方法进行转换才能得到实际的index
"			

"1、完成带有约束条件的VRP问题（20points）

2、结果正确（10points）"

In [1]:
import pandas as pd
#from lzylearn import common_tools

df = pd.read_excel('./cities.xlsx')
df.head()

,name,location
0,北京,"116.403039,39.914909"
1,天津,"117.216419,39.143831"
2,哈尔滨,"126.637622,45.766187"
3,长春,"125.330606,43.918866"
4,沈阳,"123.401674,41.800522"


In [2]:
#print(common_tools.compute_distance(1,2,2,1))

In [3]:
city_list = df['name'].values.tolist()
print(city_list)

['北京', '天津', '哈尔滨', '长春', '沈阳', '石家庄', '呼和浩特', '太原', '上海', '杭州', '福州', '广州', '海口', '合肥', '郑州', '南京', '济南', '兰州', '银川', '西宁', '西安', '拉萨', '乌鲁木齐', '成都', '重庆', '南宁', '贵阳', '昆明', '武汉', '长沙', '南昌', '香港', '澳门']


In [1]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

class tsp(object):
    def __init__(self, city_names=None, num_vehicles=1):
        
        self.df = pd.read_excel('./cities.xlsx')
        self.all_city = self.df['name'].values
        self.num_vehicles = num_vehicles
        
        if city_names is not None:
            self.city_names = city_names
            self.df = self.df[self.df['name'].isin(city_names)]
        else:
            self.city_names = self.all_city
        
    def create_data_model(self):
        data = {}
        temp = pd.read_excel('./distance.xlsx', index_col=0)
        #print(temp.head())
        temp = temp[(temp.index.isin(self.city_names))][self.city_names]
        data['distance_matrix'] = temp.values / 1000
        data['num_vehicles'] = self.num_vehicles
        data['depot'] = 0
        return data
    
    def get_solution(self, manager, routing, solution):
        print('总行驶里程: {} 公里'.format(solution.ObjectiveValue()))
        
        distance_list = []
        route_list = []
        
        for vehicle_id in range(self.num_vehicles):
            
            route_distance = 0
            route = []
            
            index = routing.Start(vehicle_id)
            
            while not routing.IsEnd(index):
                #plan_output += ' {} ->'.format(city_names[manager.IndexToNode(index)])
                index_show = manager.IndexToNode(index)
                route.append(index_show)
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
            #plan_output += city_names[manager.IndexToNode(index)]
            #print(plan_output)
            #print('{}公里'.format(route_distance))
            route_list.append(route)
            distance_list.append(route_distance)
                
        return route_list, distance_list
    
    def add_distance_dimension(self, routing, transit_callback_index) :
        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,
            10000,
            True,
            dimension_name)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)
    
    def work(self):
        data = self.create_data_model()
 
        # 创建路线管理，tsp_size（城市数量）, num_vehicles（车的数量）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
 
        # 创建 Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # 计算两点之间的距离
        def distance_callback(from_index, to_index):
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
        
        self.add_distance_dimension(routing, transit_callback_index)
        
        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # 求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # 输出结果
        #print('solution:',solution)
        route,route_distance = self.get_solution(manager, routing, solution)
           # print_solution(manager, routing, solution)
        return route, route_distance

if __name__ =='__main__': 
    
    model = tsp(num_vehicles=4)
    route, route_distance = model.work()
    #model.create_data_model()
    print(route)
    print(route_distance)

总行驶里程: 736631 公里
[[0, 7, 18, 19, 17, 22, 6], [0, 21], [0, 5, 20, 23, 24, 26, 27, 25, 12, 32, 11, 31, 30, 13, 16], [0, 14, 28, 29, 10, 9, 8, 15, 1, 4, 2, 3]]
[6341, 7096, 6749, 6845]
